In [86]:
# https://github.com/kwmcbride/kipet_examples/blob/master/examples/example_4/Ex_4_sim_aspirin.py

In [87]:
import kipet
from pyomo.core import exp
import pandas as pd
import altair as alt

In [88]:
rm = kipet.ReactionModel('reaction1')

In [89]:
# Components
SA = rm.component('SA', value=1.0714, description='Salicitilc acid')
AA = rm.component('AA', value=9.3828, description='Acetic anhydride')
ASA = rm.component('ASA', value=0.0177, description='Acetylsalicylic acid')
HA = rm.component('HA', value=0.0177, description='Acetic acid')
ASAA = rm.component('ASAA', value=0.000015, description='Acetylsalicylic anhydride')
H2O = rm.component('H2O', value=0.0, description='Water')

In [90]:
# Parameters
k0 = rm.parameter('k0', value=0.0360309)
k1 = rm.parameter('k1', value=0.1596062)
k2 = rm.parameter('k2', value=6.8032345)
k3 = rm.parameter('k3', value=1.8028763)
kd = rm.parameter('kd', value=7.1108682)
kc = rm.parameter('kc', value=0.7566864)
Csa = rm.parameter('Csa',value=2.06269996)

In [91]:
V = rm.volume(value = 0.0202)
Masa = rm.state('Masa', value = 0.0)
Msa = rm.state('Msa', value = 9.537)

In [92]:
f = rm.fixed_state('f', description = 'flow f', data = 'traj')
Csat = rm.fixed_state('Csat', description = 'C saturation', data = 'traj')

In [93]:
# Model constants
cMsa = rm.constant('cMsa', value=138.121)
cMasa = rm.constant('cMasa', value=180.157)
Cin = rm.constant('Cin', value=39.1)

In [94]:
# Stoichiometric matrix (component based)
gammas = dict()
gammas['SA']=    [-1, 0, 0, 0, 1, 0]
gammas['AA']=    [-1,-1, 0,-1, 0, 0]
gammas['ASA']=   [ 1,-1, 1, 0, 0,-1]
gammas['HA']=    [ 1, 1, 1, 2, 0, 0]
gammas['ASAA']=  [ 0, 1,-1, 0, 0, 0]
gammas['H2O']=   [ 0, 0,-1,-1, 0, 0]

In [95]:
epsilon = dict()
epsilon['SA']= 0.0
epsilon['AA']= 0.0
epsilon['ASA']= 0.0
epsilon['HA']= 0.0
epsilon['ASAA']= 0.0
epsilon['H2O']= 1.0

partial_vol = dict()
partial_vol['SA']=0.0952552311614
partial_vol['AA']=0.101672206869
partial_vol['ASA']=0.132335206093
partial_vol['HA']=0.060320218688
partial_vol['ASAA']=0.186550717015
partial_vol['H2O']=0.0883603912169

In [96]:
# Reactions
r0 = rm.add_reaction('r0', k0*SA*AA, description='Reaction 0')
r1 = rm.add_reaction('r1', k1*ASA*AA, description='Reaction 1' )
r2 = rm.add_reaction('r2', k2*ASAA*H2O, description='Reaction 2' )
r3 = rm.add_reaction('r3', k3*AA*H2O, description='Reaction 3')

In [97]:
step = 1/(1 + exp(-Msa/1e-4))
r4 = rm.add_reaction('r4', kd*(Csa - SA + 1e-6)**1.90*step, description='Reaction 4' )

diff = ASA - Csat
r5 = rm.add_reaction('r5', 0.3950206559*kc*(diff+((diff)**2+1e-6)**0.5)**1.34, description='Reaction 5' )

In [98]:
odes = rm.reactions_from_stoich(gammas, add_odes = False)

In [99]:
# Build expression for the volume
v_sum_float = 0
for com in rm.components.names:
    v_sum_float += partial_vol[com] * (odes[com] + epsilon[com]*f/V*Cin)

v_sum = rm.add_expression('v_sum', v_sum_float, description='Volume Sum')

In [100]:
# Add ODEs for the components with feeds
for com in rm.components.names:
    rm.add_ode(com, odes[com] + epsilon[com]*f/V*Cin )

In [101]:
# Add ODEs for complementary states
rm.add_ode('V', V*v_sum )
rm.add_ode('Masa', cMasa*V*r5 )
rm.add_ode('Msa', -cMsa*V*r4 )

In [102]:
rm.add_data('traj', category = 'trajectory', file = '/home/paperspace/learn_kipet/data_sets/extra_states.txt')

In [103]:
rm.add_data('conc', category = 'trajectory', file = '/home/paperspace/learn_kipet/data_sets/concentrations.txt')

In [104]:
rm.datasets['traj'].data

,Csat,Masa,Msa,T,V,f
0.0000,1.003993,0.000000,9.537000,313.886158,0.020200,0.0
0.0830,1.017234,0.000000,6.145141,314.316768,0.022549,0.0
0.1663,1.030679,0.000000,4.871626,314.749106,0.023428,0.0
0.2493,1.044221,0.000000,4.160760,315.179714,0.023918,0.0
0.3330,1.060331,0.000000,3.675226,315.685916,0.024252,0.0
...,...,...,...,...,...,...
207.7313,0.429726,10.628654,-0.000000,279.307636,0.023348,0.0
208.2312,0.429668,10.628726,-0.000000,279.285749,0.023348,0.0
208.7313,0.429694,10.628789,-0.000000,279.295486,0.023348,0.0
209.2312,0.429694,10.628848,-0.000000,279.295486,0.023348,0.0


In [105]:
rm.datasets['conc'].data

,AA,ASA,ASAA,H2O,HA,SA
0.0000,9.382756,0.017681,0.000015,0.000000,0.017743,1.071358
0.0833,8.998348,0.048488,0.000241,0.000000,0.049452,1.392765
0.1663,8.776876,0.082289,0.001075,0.000000,0.086587,1.542972
0.2500,8.615339,0.116226,0.002958,0.000000,0.128057,1.627825
0.3330,8.484006,0.148051,0.006201,0.000000,0.172856,1.679714
...,...,...,...,...,...,...
207.6483,0.000000,0.430104,0.000000,3.513000,15.200497,0.000425
208.1482,0.000000,0.430088,0.000000,3.513007,15.200531,0.000425
208.6483,0.000000,0.430074,0.000000,3.513014,15.200560,0.000425
209.1482,0.000000,0.430061,0.000000,3.513020,15.200588,0.000425


In [106]:
filename = '/home/paperspace/learn_kipet/data_sets/init_Z.csv'
rm.add_data('init_Z', category='trajectory', file=filename)

filename = '/home/paperspace/learn_kipet/data_sets/init_X.csv'
rm.add_data('init_X', category='trajectory', file=filename)

filename = '/home/paperspace/learn_kipet/data_sets/init_Y.csv'
rm.add_data('init_Y', category='trajectory', file=filename)

In [113]:
rm.datasets['init_Y'].data

,r4,r5,v_sum,r0,r1,r2,r3,f,Csat
0.000000,6.993785e+00,0.000000,0.664988,0.362209,0.026507,0.0,0.0,0.0,1.003993
0.526314,8.548702e-01,0.000000,0.082408,0.509645,0.245182,0.0,0.0,0.0,1.102280
1.052628,6.306370e-01,0.000000,0.062357,0.478677,0.336642,0.0,0.0,0.0,1.216276
1.578943,5.661196e-01,0.000000,0.056658,0.440753,0.358752,0.0,0.0,0.0,1.320088
2.105257,5.202411e-01,0.000000,0.052352,0.404662,0.351791,0.0,0.0,0.0,1.398497
...,...,...,...,...,...,...,...,...,...
208.420443,1.423935e-10,0.000057,-0.000008,0.000000,0.000000,0.0,0.0,0.0,0.429678
208.946757,1.420492e-10,0.000053,-0.000007,0.000000,0.000000,0.0,0.0,0.0,0.429694
209.473072,1.417066e-10,0.000051,-0.000007,0.000000,0.000000,0.0,0.0,0.0,0.429694
209.999386,1.413656e-10,0.000049,-0.000006,0.000000,0.000000,0.0,0.0,0.0,0.429694


In [107]:
# Simulations require a time span
rm.set_time(210.5257)

In [108]:
rm.settings.collocation.nfe = 100
rm.settings.simulator.method = 'dae.collocation'
#rm.settings.simulator.solver_opts.update({'halt_on_ampl_error' :'yes'})
rm.settings.simulator.tee = True
rm.settings.solver.linear_solver = 'ma27'

In [109]:
# Initialize the model variables with the provided data
rm.initialize_from_trajectory('Z', 'init_Z')
rm.initialize_from_trajectory('X', 'init_X')
rm.initialize_from_trajectory('Y', 'init_Y')

In [110]:
rm.simulate()

# Simulator: Setting up simulation model
# TemplateBuilder: Preparing model for simulator
# Simulator: Finished creating simulator
Ipopt 3.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:    40234
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    10509

Total number of variables............................:     7516
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     